<a href="https://colab.research.google.com/github/ismailsangu/IE423-TASK/blob/main/Task_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# JOKE RECOMMENDATION:

## Initialize

In [79]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Load Data

In [80]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [81]:
# Joke Id Data
dfJoke = pd.read_csv('/content/drive/MyDrive/Data_Sets/JokeText.csv')



# User ratings for each joke
dfRating = pd.read_csv('/content/drive/MyDrive/Data_Sets/UserRatings1.csv')


In [82]:
dfJoke.head()

,JokeId,JokeText
0,0,"A man visits the doctor. The doctor says ""I ha..."
1,1,This couple had an excellent relationship goin...
2,2,Q. What's 200 feet long and has 4 teeth? \n\nA...
3,3,Q. What's the difference between a man and a t...
4,4,Q.\tWhat's O. J. Simpson's Internet address? \...


In [83]:
dfRating.head()

,JokeId,User1,User2,User3,User4,User5,User6,User7,User8,User9,...,User36701,User36702,User36703,User36704,User36705,User36706,User36707,User36708,User36709,User36710
0,0,5.10,-8.79,-3.50,7.14,-8.79,9.22,-4.03,3.11,-3.64,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.91,NaN
1,1,4.90,-0.87,-2.91,-3.88,-0.58,9.37,-1.55,0.92,-3.35,...,NaN,NaN,NaN,-5.63,NaN,-6.07,NaN,-1.60,-4.56,NaN
2,2,1.75,1.99,-2.18,-3.06,-0.58,-3.93,-3.64,7.52,-6.46,...,NaN,NaN,NaN,NaN,NaN,4.08,NaN,NaN,8.98,NaN
3,3,-4.17,-4.61,-0.10,0.05,8.98,9.27,-6.99,0.49,-3.40,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,5.15,5.39,7.52,6.26,7.67,3.45,5.44,-0.58,1.26,...,2.28,-0.49,5.1,-0.29,-3.54,-1.36,7.48,-5.78,0.73,2.62


#RECOMMENDER SYSTEMS:

#1. Content Based Filtering

#Prepare Data

In [84]:
dfJoke.shape

(100, 2)

In [85]:
# Remove duplicates
dfJoke.drop_duplicates(subset ='JokeText', keep = 'first', inplace = True)
dfJoke.shape

(100, 2)

There is no duplicate.

In [86]:
dfJoke.isna().sum()

JokeId      0
JokeText    0
dtype: int64

There is no missed value.

In [87]:
dfRating.isna().any().any()

True

There are missed values.

In [88]:
dfRating =dfRating.fillna(method = 'ffill')

<ipython-input-88-07760ac53296>:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  dfRating =dfRating.fillna(method = 'ffill')


In [89]:
dfRating.isna().any().any()

True

There is still missed values.

In [90]:
cols_num_null = dfRating.columns[dfRating.isnull().any()].tolist()
dfRating = dfRating.drop(cols_num_null, axis=1)

In [91]:
dfRating.isna().any().any()


False

Missed values are dropped.

In [92]:
dfRating.duplicated().sum()

0

There is no duplicated value.

In [93]:
dfRating.head()

,JokeId,User1,User2,User3,User4,User5,User6,User7,User8,User9,...,User36398,User36428,User36445,User36447,User36482,User36522,User36533,User36674,User36689,User36709
0,0,5.10,-8.79,-3.50,7.14,-8.79,9.22,-4.03,3.11,-3.64,...,5.92,3.45,0.87,3.64,0.15,-2.28,5.53,-5.00,-4.51,2.91
1,1,4.90,-0.87,-2.91,-3.88,-0.58,9.37,-1.55,0.92,-3.35,...,-5.73,6.07,3.64,-1.70,-7.38,5.10,1.75,0.83,-3.06,-4.56
2,2,1.75,1.99,-2.18,-3.06,-0.58,-3.93,-3.64,7.52,-6.46,...,5.97,-9.27,3.64,-1.70,-7.18,5.10,-2.96,0.83,-3.06,8.98
3,3,-4.17,-4.61,-0.10,0.05,8.98,9.27,-6.99,0.49,-3.40,...,5.97,-9.27,3.64,-1.70,-7.18,5.10,-2.96,0.83,-3.06,8.98
4,4,5.15,5.39,7.52,6.26,7.67,3.45,5.44,-0.58,1.26,...,3.54,-9.81,-4.27,-1.70,-1.94,-2.77,-7.77,1.99,-0.44,0.73


#Build Model

In [94]:
# Generate a matrix of common terms that show up in each joke
from sklearn.feature_extraction.text import TfidfVectorizer
mdlTfvJoke = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=1, stop_words='english')
tfidf_matrix = mdlTfvJoke.fit_transform(dfJoke['JokeText'])
tfidf_matrix.shape

(100, 3774)

In [95]:
# Calculate cosine similarity between each pair of movies as a function of the similarity of the common terms

from sklearn.metrics.pairwise import cosine_similarity
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
cosine_sim.shape

(100, 100)

#Predict

In [96]:
# Prepare recommendation function (build code from scratch and then package as function for ease of understanding)

titles = dfJoke['JokeText']
indices = pd.Series(dfJoke.index, index=dfJoke['JokeText'])

def get_recommendations(JokeText):
    idx = indices[JokeText]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    joke_indices = [i[0] for i in sim_scores]
    return titles.iloc[joke_indices]

In [97]:
def get_recommendations_by_keyword(keyword, top_n=10):

    # Create a query vector for the keyword
    keyword_vector = mdlTfvJoke.transform([keyword])

    # Calculate cosine similarity between the keyword and all jokes
    similarities = cosine_similarity(keyword_vector, tfidf_matrix)

    # Get indices of jokes sorted by similarity
    sorted_indices = similarities.argsort()[0][::-1]

    # Return the top_n most similar jokes
    return dfJoke['JokeText'].iloc[sorted_indices[:top_n]]


In [98]:
get_recommendations_by_keyword("engineer", top_n=5)

46    There was an engineer who had an exceptional g...
27    A mechanical, electrical and a software engine...
48    Three engineering students were gathered toget...
92    Reaching the end of a job interview, the human...
61    A group of  managers were given the assignment...
Name: JokeText, dtype: object

In [99]:
get_recommendations_by_keyword("Turkish", top_n=5)

99    Q: Whats the difference between greeting a Que...
36    A Jewish young man was seeing a psychiatrist f...
26    Clinton returns from a vacation in Arkansas an...
27    A mechanical, electrical and a software engine...
28    An old Scotsmen is sitting with a younger Scot...
Name: JokeText, dtype: object

Thanks to Content Based Filtering,  we can take recommendation by keyword according to similarities even if the key word is not included.

### 2. Collaborative Filtering

#Prepare Data

In [100]:
dfRating.head(99)

,JokeId,User1,User2,User3,User4,User5,User6,User7,User8,User9,...,User36398,User36428,User36445,User36447,User36482,User36522,User36533,User36674,User36689,User36709
0,0,5.10,-8.79,-3.50,7.14,-8.79,9.22,-4.03,3.11,-3.64,...,5.92,3.45,0.87,3.64,0.15,-2.28,5.53,-5.00,-4.51,2.91
1,1,4.90,-0.87,-2.91,-3.88,-0.58,9.37,-1.55,0.92,-3.35,...,-5.73,6.07,3.64,-1.70,-7.38,5.10,1.75,0.83,-3.06,-4.56
2,2,1.75,1.99,-2.18,-3.06,-0.58,-3.93,-3.64,7.52,-6.46,...,5.97,-9.27,3.64,-1.70,-7.18,5.10,-2.96,0.83,-3.06,8.98
3,3,-4.17,-4.61,-0.10,0.05,8.98,9.27,-6.99,0.49,-3.40,...,5.97,-9.27,3.64,-1.70,-7.18,5.10,-2.96,0.83,-3.06,8.98
4,4,5.15,5.39,7.52,6.26,7.67,3.45,5.44,-0.58,1.26,...,3.54,-9.81,-4.27,-1.70,-1.94,-2.77,-7.77,1.99,-0.44,0.73
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,94,0.92,-1.75,-3.79,6.26,-3.45,3.74,4.17,1.21,3.74,...,-8.40,-4.56,5.10,-3.30,-6.36,-4.42,-9.81,-6.50,8.11,-9.61
95,95,6.31,-1.02,3.98,3.93,9.13,1.94,0.44,1.21,6.94,...,-8.40,-4.56,5.10,-3.30,-6.36,-4.42,-9.81,-6.50,8.11,-9.61
96,96,-4.95,-0.97,-6.46,-2.57,9.17,1.99,-0.78,5.34,5.83,...,-8.40,-4.56,5.10,-3.30,-6.36,-4.42,-0.39,-6.50,8.11,-9.61
97,97,-0.19,4.13,-6.89,1.07,9.17,3.45,-1.02,1.94,5.53,...,-8.40,-4.56,5.10,-3.30,-6.36,-4.42,-0.39,-6.50,8.11,-9.61


In [101]:
dfRating.columns

Index(['JokeId', 'User1', 'User2', 'User3', 'User4', 'User5', 'User6', 'User7',
       'User8', 'User9',
       ...
       'User36398', 'User36428', 'User36445', 'User36447', 'User36482',
       'User36522', 'User36533', 'User36674', 'User36689', 'User36709'],
      dtype='object', length=31827)

In [102]:
df_long = pd.melt(dfRating, id_vars=["JokeId"], var_name="UserId", value_name="Rating")
df_long['UserId'] = df_long['UserId'].str.extract('(\d+)').astype(int)
# Display the transformed dataframe
df_long.head()

,JokeId,UserId,Rating
0,0,1,5.10
1,1,1,4.90
2,2,1,1.75
3,3,1,-4.17
4,4,1,5.15


In [103]:
df_long[df_long['UserId'] == 1]

,JokeId,UserId,Rating
0,0,1,5.10
1,1,1,4.90
2,2,1,1.75
3,3,1,-4.17
4,4,1,5.15
...,...,...,...
95,95,1,6.31
96,96,1,-4.95
97,97,1,-0.19
98,98,1,3.25


#Build Model

In [106]:
# Prepare data into Surprise library format

!pip3 install scikit-surprise #or !conda install -c conda-forge scikit-surprise
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import train_test_split

reader = Reader(rating_scale=(0,5))
# Load the data into Surprise format
X = Dataset.load_from_df(df_long[['JokeId','UserId','Rating']], reader)
X_train, X_test = train_test_split(X, test_size=.25)

  Using cached scikit_surprise-1.1.4.tar.gz (154 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357288 sha256=adaa6d79f87ca37bd1869d76a5decf441e791183290acf011d1cdfd1eec19966
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


In [107]:
# Define SVD model

from surprise import SVD

mdlSvdRating = SVD()

In [108]:
# Fit SVD model

mdlSvdRating.fit(X_train)
test_pred = mdlSvdRating.test(X_test)

In [109]:
# Evalute SVD accuracy

from surprise import accuracy

accuracy.rmse(test_pred)

RMSE: 4.5341


4.5341199535580445

RMSE: 4.5411 is quite high for a recommendation so we can improve accuracy by hyperparameter tuning.

In [110]:
# Tune hyperparameters

from surprise.model_selection import GridSearchCV

param_grid = {'n_epochs': [10], 'lr_all': [0.002],
              'reg_all': [0.4]}
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=2)

gs.fit(X)

# best RMSE score
print(gs.best_score['rmse'])

# combination of parameters that gave the best RMSE score
print(gs.best_params['rmse'])

4.696956142673633
{'n_epochs': 10, 'lr_all': 0.002, 'reg_all': 0.4}


Because of long running time, lower paramater grid is chosen.

In [111]:
# Cross-validate

from surprise.model_selection import cross_validate

cross_validate(mdlSvdRating, X, measures=['RMSE', 'MAE'], cv=3, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    4.5437  4.5346  4.5406  4.5396  0.0037  
MAE (testset)     3.5772  3.5713  3.5752  3.5746  0.0024  
Fit time          85.60   86.43   92.12   88.05   2.90    
Test time         21.71   15.75   19.08   18.85   2.44    


{'test_rmse': array([4.54365134, 4.53463631, 4.54064474]),
 'test_mae': array([3.57715203, 3.57132687, 3.57520692]),
 'fit_time': (85.59701657295227, 86.42617058753967, 92.12301874160767),
 'test_time': (21.711856603622437, 15.750828742980957, 19.083291053771973)}

#Predict

In [140]:
df_long[df_long['JokeId'] == 0]

,JokeId,UserId,Rating
0,0,1,5.10
100,0,2,-8.79
200,0,3,-3.50
300,0,4,7.14
400,0,5,-8.79
...,...,...,...
3182100,0,36522,-2.28
3182200,0,36533,5.53
3182300,0,36674,-5.00
3182400,0,36689,-4.51


In [139]:
mdlSvdRating.predict(0, 5)

Prediction(uid=0, iid=5, r_ui=None, est=1.0681158438530602, details={'was_impossible': False})

According to other similar users' rate to JokeId=0, UserId=5 will rate 1.068 to JokeId=0.

#Conclusion



*   After the data cleaning,to make Content Based Filtering, we can take joke recommendation by keyword according to similarities even if the key word is not included by using TF-IDF Vectorization.
* To make Collaborative Filtering, data trained in SVD Model and made predictions of user joke rating according to other users ratings.

